<a href="https://colab.research.google.com/github/KubX44/Tp-Spark-SQL/blob/main/TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sous Google Colab, installer pyspark :

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9ca8827e57f240b089171b4ba633fe363382bad3f013dc500c029bdfcd66a5fc
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


1) Créer un Dataframe à partir du fichier ngram.csv :

In [8]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql import Row

appname = "tp3"
config = SparkConf().setAppName(appname)
sparkcontext = SparkContext.getOrCreate();
sqlContext = SQLContext(sparkcontext)

lines = sparkcontext.textFile("/content/ngram.csv")
parts = lines.map(lambda l: l.split("\t"))
ngrams = parts.map(lambda x: Row(ngram=x[0], year=int(x[1]), count=int(x[2]), pages=int(x[3]), books=int(x[4])))
schema = sqlContext.createDataFrame(ngrams)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


2) Enregistrer le Dataframe créé en tant que table temporaire.


In [9]:
schema.registerTempTable("ngrams")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


3.1) Retourner tous les bi-grammes dont le nombre Count est supérieur à cinq.

In [11]:
#SQL :
req = sqlContext.sql("SELECT ngram FROM ngrams WHERE count > 5")
req.show()

+--------+
|   ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



In [13]:
#l’API SparkSQL :
req = schema.filter(schema['count'] > 5).select('ngram')
req.show()

+--------+
|   ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
|    ! 09|
+--------+



3.2) Retourner le nombre total de bi-grammes dans chaque année.

In [14]:
#SQL :
req = sqlContext.sql("SELECT year, COUNT(*) AS cnt FROM ngrams GROUP BY year")
req.show()

+----+---+
|year|cnt|
+----+---+
|1806|  1|
|1840|  1|
|1882|  1|
|2007|  2|
|1887|  1|
|1991|  2|
|1858|  1|
|1833|  1|
|1831|  1|
|1987|  2|
|1999|  2|
|1997|  2|
|1884|  1|
|1845|  1|
|1877|  1|
|1829|  1|
|1867|  1|
|2006|  2|
|1876|  1|
|1879|  1|
+----+---+
only showing top 20 rows



In [15]:
#l’API SparkSQL :
req = schema.groupBy("year").count()
req.show()

+----+-----+
|year|count|
+----+-----+
|1806|    1|
|1840|    1|
|1882|    1|
|2007|    2|
|1887|    1|
|1991|    2|
|1858|    1|
|1833|    1|
|1831|    1|
|1987|    2|
|1999|    2|
|1997|    2|
|1884|    1|
|1845|    1|
|1877|    1|
|1829|    1|
|1867|    1|
|2006|    2|
|1876|    1|
|1879|    1|
+----+-----+
only showing top 20 rows



3.3) Retourner les bi-grammes qui ont le plus grand nombre de count dans chaque
année.

In [16]:
#SQL :
req = sqlContext.sql("SELECT ngram, year, MAX(count) AS maxCount FROM ngrams GROUP BY ngram, year")
req.show()

+--------+----+--------+
|   ngram|year|maxCount|
+--------+----+--------+
|    ! 09|1880|       2|
|! $17.95|2008|       2|
|    ! 09|1845|       1|
|! $17.95|1995|       1|
|! $17.95|2004|      14|
|    ! 09|1780|       1|
|    ! 09|1833|       1|
|! $17.95|2007|       2|
|    ! 09|1872|       1|
|    ! 09|1861|       1|
|    ! 09|1806|       1|
|    ! 09|1882|       2|
|! $17.95|2001|       5|
|    ! 09|1881|       3|
|    ! 09|1884|       5|
|    ! 09|1823|       1|
|    ! 09|1848|       1|
|    ! 09|1887|       1|
|    ! 09|1803|       1|
|    ! 09|1877|       2|
+--------+----+--------+
only showing top 20 rows



In [18]:
#l’API SparkSQL :
req = schema.groupBy("ngram", "Year").max("count")
req.show()

+--------+----+----------+
|   ngram|Year|max(count)|
+--------+----+----------+
|    ! 09|1880|         2|
|! $17.95|2008|         2|
|    ! 09|1845|         1|
|! $17.95|1995|         1|
|! $17.95|2004|        14|
|    ! 09|1780|         1|
|    ! 09|1833|         1|
|! $17.95|2007|         2|
|    ! 09|1872|         1|
|    ! 09|1861|         1|
|    ! 09|1806|         1|
|    ! 09|1882|         2|
|! $17.95|2001|         5|
|    ! 09|1881|         3|
|    ! 09|1884|         5|
|    ! 09|1823|         1|
|    ! 09|1848|         1|
|    ! 09|1887|         1|
|    ! 09|1803|         1|
|    ! 09|1877|         2|
+--------+----+----------+
only showing top 20 rows



3.4) Retourner tous les bi-grammes qui sont apparus dans 20 années différentes

In [19]:
#SQL :
req = sqlContext.sql("SELECT ngram FROM ngrams GROUP BY ngram HAVING COUNT(DISTINCT Year) = 20").show()

+--------+
|   ngram|
+--------+
|! $17.95|
+--------+



In [20]:
#l’API SparkSQL :
result = schema.groupBy("ngram").agg(
    countDistinct("Year").alias("NumYears")
).filter("NumYears = 20").select("ngram").show()

+--------+
|   ngram|
+--------+
|! $17.95|
+--------+



3.5) Retourner tous les bi-grammes qui contiennent le caractère ‘!’ dans la première
partie et le caractère ‘9’ dans la deuxième partie (les deux parties sont séparées par
un espace).

In [21]:
#SQL :
sqlContext.sql("SELECT distinct ngram FROM ngrams WHERE ngram LIKE '%!% %' AND ngram LIKE '% %9%'").show()

+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



In [22]:
#l’API SparkSQL :
schema.filter((col("ngram").like("%!% %")) & (col("ngram").like("% %9%"))).select("ngram").distinct().show()

+--------+
|   ngram|
+--------+
|    ! 09|
|! $17.95|
+--------+



3.6) Retourner les bi-grammes qui sont apparus dans toutes les années présentes
dans les données.

In [23]:
#SQL :
sqlContext.sql("SELECT ngram FROM ngrams GROUP BY ngram HAVING COUNT(DISTINCT Year) = (SELECT COUNT(DISTINCT Year) FROM ngrams)").show()

+-----+
|ngram|
+-----+
| ! 09|
+-----+



In [24]:
#l’API SparkSQL :
distinct_years = schema.select("Year").distinct().count()

result = schema.groupBy("ngram").agg(countDistinct("Year").alias("distinct_years")) \
    .filter("distinct_years = {}".format(distinct_years)) \
    .select("ngram").show()

+-----+
|ngram|
+-----+
| ! 09|
+-----+



3.7) Retourner le nombre total de pages et de livres dans lesquels chaque bigramme apparaît pour chaque année disponible, trié par ordre alphabétique.

In [25]:
#SQL :
sqlContext.sql("SELECT Year, ngram, SUM(Pages) AS TotalPages, SUM(Books) AS TotalBooks FROM ngrams GROUP BY Year, ngram ORDER BY ngram ASC").show()

+----+--------+----------+----------+
|Year|   ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1998|! $17.95|         3|         3|
|1992|! $17.95|         5|         5|
|1995|! $17.95|         1|         1|
|1997|! $17.95|         5|         5|
|1993|! $17.95|         2|         2|
|1987|! $17.95|         1|         1|
|1985|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|2004|! $17.95|        14|        14|
|1996|! $17.95|         2|         2|
|2005|! $17.95|        13|        13|
|2001|! $17.95|         4|         4|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        10|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|2000|! $17.95|         9|         9|
|2002|! $17.95|         5|         5|
+----+--------+----------+----------+
only showing top 20 rows



In [27]:
#l’API SparkSQL :
result = (schema
    .groupBy("Year", "ngram")
    .agg(sum("Pages").alias("TotalPages"), sum("Books").alias("TotalBooks"))
    .orderBy("ngram")
)
result.show()

+----+--------+----------+----------+
|Year|   ngram|TotalPages|TotalBooks|
+----+--------+----------+----------+
|1998|! $17.95|         3|         3|
|1992|! $17.95|         5|         5|
|1995|! $17.95|         1|         1|
|1997|! $17.95|         5|         5|
|1993|! $17.95|         2|         2|
|1987|! $17.95|         1|         1|
|1985|! $17.95|         1|         1|
|1991|! $17.95|         1|         1|
|1990|! $17.95|         1|         1|
|2007|! $17.95|         2|         2|
|2004|! $17.95|        14|        14|
|1996|! $17.95|         2|         2|
|2005|! $17.95|        13|        13|
|2001|! $17.95|         4|         4|
|2003|! $17.95|         2|         2|
|1999|! $17.95|        10|        10|
|2006|! $17.95|         5|         5|
|2008|! $17.95|         2|         2|
|2000|! $17.95|         9|         9|
|2002|! $17.95|         5|         5|
+----+--------+----------+----------+
only showing top 20 rows

